
# Note book for the project 2 

Kaggle competition link: [Submition]('https://www.kaggle.com/c/epfml17-text/submit')

## Pipeline: 


### Create cooc matrix

1. sh build_vocab.sh
2. sh cut_vocab.sh
3. python3 pickle_vocab.py
4. python3 cooc.py

Now given the co-occurrence matrix and the vocabulary, it is not hard to train GloVe word embeddings, that is to compute an embedding vector for wach word in the vocabulary. We suggest to implement SGD updates to train the matrix factorization, as in

5. python3 glove_template.py

Once you tested your system on the small set of 10% of all tweets, we suggest you run on the full datasets pos_train_full.txt, neg_train_full.txt

### Building a Text Classifier:

1. Construct Features for the Training Texts: Load the training tweets and the built GloVe word embeddings. Using the word embeddings, construct a feature representation of each training tweet (by averaging the word vectors over all words of the tweet).

2. Train a Linear Classifier: Train a linear classifier (e.g. logistic regression or SVM) on your constructed features, using the scikit learn library, or your own code from the earlier labs. Recall that the labels indicate if a tweet used to contain a 🙂 or 🙁 smiley.

3. Prediction: Predict labels for all tweets in the test set.

4. Submission / Evaluation: Submit your predictions to kaggle, and verify the obtained misclassification error score. (You can also use a local separate validation set to get faster feedback on the accuracy of your system). Try to tune your system for best evaluation score.

### Extensions:
Naturally, there are many ways to improve your solution, both in terms of accuracy and computation speed. More advanced techniques can be found in the recent literature.


TODO :

- implement cross-validation
- 

Importing usefull library

In [7]:
import numpy as np
import pandas as pd
%matplotlib inline
#!/usr/bin/env python3
from scipy.sparse import *
from sklearn import linear_model
import numpy as np
import pickle
import random

# 0.Variables


In [2]:
file_storing_ts = 'embeddings.npy'
file_storing_pos_ts_tweets = 'train_pos.txt'
file_storing_neg_ts_tweets = 'train_neg.txt'
file_storing_te_tweets = 'test_data.txt'
nb_dim = 20



In [24]:
def word_embeddings(file_name='cooc.pkl', destination='embeddings.npy'):
    print("loading cooccurrence matrix")
    with open(file_name, 'rb') as f:
        cooc = pickle.load(f)
    print("{} nonzero entries".format(cooc.nnz))
    
    nmax = 100
    print("using nmax =", nmax, ", cooc.max() =", cooc.max())
    print("initializing embeddings")
    embedding_dim = 20
    xs = np.random.normal(size=(cooc.shape[0], embedding_dim))
    ys = np.random.normal(size=(cooc.shape[1], embedding_dim))
    eta = 0.001
    alpha = 3 / 4
    epochs = 10
    
    for epoch in range(epochs):
        print("epoch {}".format(epoch))
        for ix, jy, n in zip(cooc.row, cooc.col, cooc.data):
            
            f = ((n / nmax)**alpha) if n < nmax else 1
            inter_cost = (xs[ix]@(ys[jy]) - np.log(n))
            # We compute the gradients for both context and main vector words
            grad_main = f * inter_cost * ys[jy]
            grad_context = f * inter_cost * xs[ix]
    
            # Update the vector words
            xs[ix] = xs[ix] - (eta * grad_main)
            ys[jy] = ys[jy] - (eta * grad_context)
            
    np.save(destination, xs)
 


# 1.Construct words_embeddings for training set


In [ ]:
word_embeddings()


# 2.Load words for training set


In [65]:
with open('vocab.pkl', 'rb') as f :
    vocab = pickle.load(f)
xs = np.load(file_storing_ts)
vocab

{'<user>': 0,
 '!': 1,
 'i': 2,
 'the': 3,
 '.': 4,
 ',': 5,
 'to': 6,
 'you': 7,
 '(': 8,
 '<url>': 9,
 'a': 10,
 '...': 11,
 'and': 12,
 'my': 13,
 'me': 14,
 'of': 15,
 '?': 16,
 'is': 17,
 'for': 18,
 'in': 19,
 'it': 20,
 '"': 21,
 'this': 22,
 'so': 23,
 '-': 24,
 'with': 25,
 'on': 26,
 'that': 27,
 ')': 28,
 'be': 29,
 "i'm": 30,
 'have': 31,
 ':': 32,
 'but': 33,
 'just': 34,
 'rt': 35,
 'love': 36,
 'your': 37,
 'all': 38,
 'not': 39,
 'was': 40,
 'at': 41,
 'are': 42,
 '..': 43,
 'like': 44,
 '/': 45,
 'get': 46,
 'up': 47,
 'frame': 48,
 '&': 49,
 'lol': 50,
 'know': 51,
 'good': 52,
 'do': 53,
 'u': 54,
 'now': 55,
 'when': 56,
 'one': 57,
 'if': 58,
 'we': 59,
 'follow': 60,
 'no': 61,
 'can': 62,
 'what': 63,
 'go': 64,
 "don't": 65,
 'out': 66,
 'x': 67,
 'will': 68,
 'day': 69,
 "'": 70,
 'please': 71,
 'from': 72,
 'see': 73,
 'too': 74,
 'want': 75,
 'there': 76,
 'back': 77,
 "it's": 78,
 'today': 79,
 'about': 80,
 'really': 81,
 'how': 82,
 'got': 83,
 'thanks': 8


# 3.Construct features for training tweets


In [4]:
pos_tweets = open(file_storing_pos_ts_tweets, 'r').readlines() 
neg_tweets = open(file_storing_neg_ts_tweets, 'r').readlines()


#Construct features for positive tweets
pos_features = []

for line in pos_tweets:
    sum_w = np.zeros(nb_dim)
    count = 0
    for word in line.split():
        local_w = vocab.get(word, -1)
        if local_w != -1:
            count += 1
            sum_w += xs[local_w]
     
    # If no match between the vocab and the tweet, we set the feature vector to zeros (zero best value ?)
    if(count == 0):
        count = 1
    pos_features.append(sum_w/count)
pos_features = np.array(pos_features)    
    
#Construct features for negative tweets
neg_features = []

for line in neg_tweets:
    sum_w = np.zeros(nb_dim)
    count = 0
    for word in line.split():
        local_w = vocab.get(word, -1)
        if local_w != -1:
            count += 1
            sum_w += xs[local_w]
     
    # If no match between the vocab and the tweet, we set the feature vector to zeros (zero best value ?)
    if(count == 0):
        count = 1
    neg_features.append(sum_w/count)
    
neg_features = np.array(neg_features)


# Train the linear classifier


In [78]:
clf = linear_model.SGDClassifier(n_iter=50, warm_start=True);
X = np.concatenate((pos_features, neg_features))
scaler = preprocessing.StandardScaler()
Y = np.concatenate((np.ones(len(pos_features)), np.full(len(neg_features), -1)))
X = scaler.fit_transform(X, Y)
clf = clf.fit(X, Y)

                   

In [76]:
prediction = clf.predict(X)


In [77]:
print("accuracy : ", (1 - (np.sum(np.abs(Y-prediction))/(2*200000))) * 100, "%")

accuracy :  61.4925 %


In [14]:
Y.shape

(200000,)